# Preparação do ambiente

## Bibliotecas

In [1]:
import pandas as pd
import re
import warnings

from datetime import datetime
from datetime import timedelta

from pathlib import Path

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

## Constantes e funções auxiliares

In [3]:
ISSUES_FOLDER = Path(r'Z:\anatel\inova-fiscaliza\dados-pacp\datasets\inspecao_ecommerce\issues')

# Carga e prepação dos dados

In [7]:
files_issues = [file for file in ISSUES_FOLDER.rglob('*.csv')]

df_issues = pd.concat([pd.read_csv(file,sep=';',quotechar="'") for file in files_issues])
df_issues['Qual foi a data da coleta?'] = pd.to_datetime(df_issues['Qual foi a data da coleta?'],format='mixed')
df_issues['Qual foi a data da coleta?'] = df_issues['Qual foi a data da coleta?'].dt.date
df_issues['Arquivo PDF'] = df_issues['Descrição'].apply(
    lambda descricao: re.search('https.+pdf',descricao).group() 
    if re.search('https.+pdf',descricao) 
    else None
)
df_issues['nome'] = df_issues['Descrição'].apply(lambda descricao: descricao[3:].split('\n')[0])

df_issues.head()

,url,Projeto,Tipo,Situação,Título,Descrição,Data de início,Data limite,Atribuído para,Unidade de Lotação,...,Valor,Existe campo código SCH?,Código SCH foi fornecido?,Qual código SCH fornecido?,Código SCH é o do produto?,Código EAN foi fornecido?,Qual código EAN fornecido?,Código EAN é o do produto?,Arquivo PDF,nome
0,https://www.amazon.com.br/Smartphone-INFINIX-C...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000001,"h1. Smartphone INFINIX SMART 7, 3GB RAM 64GB C...",2024-07-08,15/07/2024,werther,GR03,...,719.10,Sim,Sim,5.136232e+10,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,"Smartphone INFINIX SMART 7, 3GB RAM 64GB Câme..."
1,https://www.amazon.com.br/Samsung-Galaxy-A32-I...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000002,"h1. Samsung Galaxy A32 Preto, com Tela Infinit...",2024-07-08,15/07/2024,yoshida,GR03,...,1799.00,Sim,Sim,4.242101e+09,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,"Samsung Galaxy A32 Preto, com Tela Infinita d..."
2,https://www.amazon.com.br/Samsung-Bateria-6000...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000003,h1. Samsung Galaxy M34 5G 128GB 6GB RAM Mega B...,2024-07-08,15/07/2024,werther,GR03,...,1149.00,Sim,Sim,9.583230e+10,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,Samsung Galaxy M34 5G 128GB 6GB RAM Mega Bate...
3,https://www.amazon.com.br/Protetora-Volta-Caso...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000004,h1. Protetora TPU Volta Caso Para Sams Galaxy ...,2024-07-08,15/07/2024,yoshida,GR03,...,59.00,Não,Não,NaN,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,Protetora TPU Volta Caso Para Sams Galaxy S7 ...
4,https://www.amazon.com.br/Smartphone-Xiaomi-Re...,Rotulagem de Dados,Fiscalização de Marketplace,Responder pesquisa,INSP_PACP_20240708_000005,h1. Smartphone Xiaomi Redmi Note 11S 6GB RAM 1...,2024-07-08,15/07/2024,werther,GR03,...,1228.99,Sim,Sim,2.119221e+10,NaN,Não,NaN,NaN,https://anatel365.sharepoint.com/:b:/r/sites/D...,Smartphone Xiaomi Redmi Note 11S 6GB RAM 128G...


## Resultados da raspagem dos marketplaces

In [8]:
map_columns = {
    'Item': 'item',
    'Texto da Busca' : 'texto_busca',
    'Arquivo da Página' : 'arquivo_pagina',
    'Data da Análise' : 'data_coleta',
    'Data da Coleta' : 'data_coleta',
    'Endereço eletrônico (URL)' : 'url',
    'Descrição do produto no anúncio' : 'descricao_produto',
    'O tipo do produto é de uso proibido no Brasil?' : 'produto_proibido',
    'O produto é passível de homologação na Anatel? (Sim ou Não)' : 'produto_passivel',
    'Tipo do Produto' : 'tipo_produto',
    'Fabricante' : 'fabricante',
    'Modelo do Produto' : 'modelo',
    'Valor do Produto (R$)' : 'valor',
    'Número de Unidades à Venda' : 'unidades_venda',
    'Existe o campo código de homologação no anúncio? (Sim ou Não)' : 'existe_campo_codigo_sch',
    'O código de homologação foi fornecido? (Sim ou Não)' : 'codigo_sch_foi_fornecido',
    'O produto é homologado? (Sim, Não e N.A.)' : 'produto_homologado',
    'Validação do código de homologação - O código de homologação fornecido é o do produto anunciado? (Sim, Não, N.A.)' : 'codigo_sch_corresponde_produto',
    'O código EAN foi fornecido? (Sim ou Não) - Apenas para Smartphones.' : 'codigo_ean_foi_fornecido',
    'O código EAN fornecido corresponde ao produto? (Sim ou Não) -  Apenas para Smartphones.' : 'codigo_ean_corresponde_produto',
    'Observação 1 - Qual Código de Homologação fornecido no anúncio?' : 'qual_codigo_sch_fornecido',
    'Marketplace' : 'marketplace',
    'Página': 'pagina',
    'Arquivo': 'arquivo_pagina',
    'Data' : 'data_coleta',
    'URL' : 'url',
    'Título': 'descricao_produto',
    'Categoria': 'tipo_produto',
    'Modelo': 'modelo',
    'Preço': 'valor',
    'Unidades à Venda': 'unidades_venda',
    'Existe campo código SCH?' : 'existe_campo_codigo_sch',
    'Código SCH foi fornecido?' : 'codigo_sch_foi_fornecido',
    'Código de Homologação': 'qual_codigo_sch_fornecido',
    'Código SCH é o do produto?': 'codigo_sch_corresponde_produto',
    'O produto é homologado?': 'produto_homologado',
    'O código EAN foi fornecido?': 'codigo_ean_foi_fornecido',
    'Código EAN é o do produto?': 'codigo_ean_corresponde_produto',
    'Código EAN-GTIN': 'qual_codigo_ean_fornecido'
}

In [9]:
results_folder = root_path / 'Resultados'
result_files = [file for file in results_folder.rglob('*.xlsx') if actual_collect_date in file.name]

map_marketplace = {
    'amazon': 'Amazon', 
    'ml': 'Mercado Livre',
    'carrefour': 'Carrefour', 
    'magalu': 'Magazine Luiza',
    'shopee': 'Shopee',
    'americanas': 'Americanas',
    'casasbahia': 'Casas Bahia'
}

map_tipo_produto = {
    'Celulares e Smartphones': 'Celulares e Smartphones',
    'smartphone': 'Celulares e Smartphones',
    'Smartphones': 'Celulares e Smartphones'
}

scrape_results = []

for file in result_files:
    
    file_name = file.name
    marketplace_code = file_name.split('_')[0]
    marketplace = map_marketplace[marketplace_code]

    df = pd.read_excel(file,dtype='str')
    # remove os caracteres R$ 
    df['Preço'] = df['Preço'].apply(lambda x: str(x).replace(r'R$','').strip())
    # remove '.' separador de milhares e troca separador de decimais de ',' para '.'
    df['Preço'] = df['Preço'].apply(lambda x: x.replace('.','').replace(',','.') if ',' in x else x)
    
    df['Marketplace'] = marketplace
    
    scrape_results.append(df)

df_produtos = pd.concat(scrape_results)

df_produtos = df_produtos.reset_index(drop=True)
df_produtos.columns = df_produtos.columns.map(map_columns)
df_produtos['tipo_produto'] = df_produtos['tipo_produto'].map(map_tipo_produto)

columns_to_keep = ['tipo_produto', 'texto_busca', 'data_coleta', 'marketplace',
                   'arquivo_pagina', 'url', 'descricao_produto',
                   'fabricante', 'modelo', 'valor', 
                   'existe_campo_codigo_sch', 'codigo_sch_foi_fornecido', 'qual_codigo_sch_fornecido', 'codigo_sch_corresponde_produto',
                   'codigo_ean_foi_fornecido', 'qual_codigo_ean_fornecido', 'codigo_ean_corresponde_produto']
df_produtos = df_produtos[columns_to_keep]

df_produtos = df_produtos.merge(df_homologados,on='qual_codigo_sch_fornecido',how='left')
df_produtos = df_produtos.merge(df_sch_ean,on='qual_codigo_sch_fornecido',how='left')
df_produtos['info_ean_homologados'].fillna('EAN indisponível',inplace=True)
df_produtos['info_modelo_ean'].fillna('EAN indisponível',inplace=True)
df_produtos = df_produtos.fillna('')

valor_pattern = '[0-9]+\.?(?:[0-9]{2})?'
df_produtos['valor'] = df_produtos['valor'].apply(lambda x: x if re.match(valor_pattern,x) else -1).astype(float)

# strip pipe caracter from the end of string
for col in ['info_tipo_produto', 'info_modelos_certificado', 'info_ean_homologados']:
    df_produtos[col] = df_produtos[col].apply(lambda x: x[:-3] if x[-3:]==' | ' else x)

df_produtos

NameError: name 'root_path' is not defined

# Análise

In [ ]:
map_fiscais_marketplace_even = {
    'Amazon': {'Atribuído para': 'werther', 'Unidade de Lotação': 'GR03'}, 
    'Mercado Livre': {'Atribuído para': 'costalo', 'Unidade de Lotação': 'UO021'},
    'Carrefour': {'Atribuído para': 'marcosrg', 'Unidade de Lotação': 'UO072'}, 
    'Magazine Luiza': {'Atribuído para': 'caldeira', 'Unidade de Lotação': 'GR04'},
    'Shopee': {'Atribuído para': 'lribeiro', 'Unidade de Lotação': 'GR02'},
    'Americanas': {'Atribuído para': 'humbertovinagre', 'Unidade de Lotação': 'GR01'},
    'Casas Bahia': {'Atribuído para': 'leandrorb', 'Unidade de Lotação': 'GR07'},
}

map_fiscais_marketplace_odd = {
    'Amazon': {'Atribuído para': 'yoshida', 'Unidade de Lotação': 'GR03'}, 
    'Mercado Livre': {'Atribuído para': 'josias', 'Unidade de Lotação': 'UO021'},
    'Carrefour': {'Atribuído para': 'denerreis', 'Unidade de Lotação': 'UO072'}, 
    'Magazine Luiza': {'Atribuído para': 'marcelo', 'Unidade de Lotação': 'GR04'},
    'Shopee': {'Atribuído para': 'danilo', 'Unidade de Lotação': 'GR02'},
    'Americanas': {'Atribuído para': 'beraldi', 'Unidade de Lotação': 'GR01'},
    'Casas Bahia': {'Atribuído para': 'parazi', 'Unidade de Lotação': 'FISF'},
}


data_inicio = datetime.strptime(DATA_COLETA,'%d/%m/%Y')
data_fim = data_inicio + timedelta(days=7)
data_fim = data_fim.strftime('%d/%m/%Y')

df_fiscaliza = df_produtos.copy()

# título da última tarefa no Fiscaliza
# as novas terão numeração sequencial a partir desta
fiscaliza_last_issue = 0
format_title = lambda i: f'INSP_PACP_{actual_collect_date}_{str(i+1+fiscaliza_last_issue).zfill(6)}'

df_fiscaliza['Projeto'] = 'Rotulagem de Dados'

df_fiscaliza['Título'] = df_fiscaliza.index.map(format_title)
df_fiscaliza['Descrição'] = df_fiscaliza.apply(lambda row: format_issue(row),axis=1)
df_fiscaliza['Tipo'] = 'Fiscalização de Marketplace'
df_fiscaliza['Situação'] = 'Responder pesquisa'
df_fiscaliza['Data de início'] = data_inicio
df_fiscaliza['Data limite'] = data_fim

# fiscais e unidade de lotação
# df_fiscaliza = df_fiscaliza.join(pd.DataFrame(df_fiscaliza['marketplace'].map(map_fiscais_marketplace).to_list()))
idx = df_fiscaliza.index.to_list()
even_idx = idx[0::2]
odd_idx = idx[1::2]
df_fiscais = pd.DataFrame(df_fiscaliza['marketplace'])
df_fiscais['fiscal_even'] = df_fiscais.iloc[even_idx]['marketplace'].map(map_fiscais_marketplace_even)
df_fiscais['fiscal_odd'] = df_fiscais.iloc[odd_idx]['marketplace'].map(map_fiscais_marketplace_odd)
df_fiscais['fiscal'] = df_fiscais[['fiscal_even','fiscal_odd']].apply(lambda row: row['fiscal_even'] if pd.isna(row['fiscal_odd']) else row['fiscal_odd'],axis=1) 
df_fiscais = pd.DataFrame(df_fiscais['fiscal'].to_list())
df_fiscaliza = df_fiscaliza.join(df_fiscais)

columns_to_issues = ['url', 'Projeto', 'Tipo', 'Situação', 'Título', 'Descrição', 
                     'Data de início', 'Data limite', 'Atribuído para', 'Unidade de Lotação', 
                     'tipo_produto', 'texto_busca', 'data_coleta', 'marketplace', 
                     'fabricante', 'modelo', 'valor', 
                     'existe_campo_codigo_sch', 'codigo_sch_foi_fornecido', 'qual_codigo_sch_fornecido', 'codigo_sch_corresponde_produto', 
                     'codigo_ean_foi_fornecido', 'qual_codigo_ean_fornecido', 'codigo_ean_corresponde_produto']
df_fiscaliza = df_fiscaliza[columns_to_issues]

columns_to_rename = ['url', 'Projeto', 'Tipo', 'Situação', 'Título', 'Descrição', 
                     'Data de início', 'Data limite', 'Atribuído para', 'Unidade de Lotação',
                     'Qual o tipo do produto? ','Texto da busca','Qual foi a data da coleta?','Marketplace',
                     'Fabricante','Modelo do Produto','Valor',
                     'Existe campo código SCH?','Código SCH foi fornecido?','Qual código SCH fornecido?','Código SCH é o do produto?',
                     'Código EAN foi fornecido?','Qual código EAN fornecido?','Código EAN é o do produto?']
df_fiscaliza.columns = columns_to_rename


# for collect_date in df_fiscaliza['Qual foi a data da coleta?'].unique():
    
#     issues_file_date = datetime.strptime(collect_date,'%d/%m/%Y').strftime('%Y%m%d')
#     issues_output_folder = root_path / 'issues'
#     issues_file_name = issues_output_folder/f'issues_{issues_file_date}.csv'
#     if not issues_file_name.exists():
#         df_fiscaliza[df_fiscaliza['Qual foi a data da coleta?']==collect_date].to_csv(issues_file_name,index=False,sep=";",quotechar="'")
#     else:
#         print(f'Issues file for collect date {collect_date} already exists. Not saved.')

issues_output_folder = root_path / 'issues'
issues_file_name = issues_output_folder/f'issues_{actual_collect_date}.csv'
if not issues_file_name.exists():
    df_fiscaliza[df_fiscaliza['Qual foi a data da coleta?']==collect_date].to_csv(issues_file_name,index=False,sep=";",quotechar="'")
else:
    print(f'Issues file for collect date {collect_date} already exists. Not saved.')

df_fiscaliza

# Resultados da raspagem do Regulatron

In [ ]:
file_regulatron = '../datasets/inspecao_ecommerce/avaliacao_regulatron/Anúncios Regulatron Amazon.xlsx'
df_regulatron = pd.read_excel(file_regulatron)
df_regulatron = df_regulatron.sort_values(by='data',ascending=False)
df_regulatron = df_regulatron.drop_duplicates(subset='url')
df_regulatron

# Resultados da segunda fiscalização

In [ ]:
file_resultados = '../datasets/inspecao_ecommerce/avaliacao_regulatron/closed_issues.csv'
df_resultados = pd.read_csv(file_resultados,sep=';')
# df_resultados = df_resultados[
#     (df_resultados['Marketplace']=='Amazon')
#     &(df_resultados['Texto da busca']=='smartphone')
# ]

# columns_to_keep = ['Título', 'url', 'Arquivo PDF']
# df_issues[df_issues['Marketplace']=='Amazon'][columns_to_keep].to_excel('../datasets/inspecao_ecommerce/avaliacao_regulatron/issues_screenshots.xlsx')
# df_resultados = df_resultados.merge(
#     df_issues[df_issues['Marketplace']=='Amazon'][columns_to_keep],
#     how='left'
# )
df_resultados

# Anúncios comuns

In [ ]:
df_anuncios_analise = df_resultados.merge(df_regulatron['url'])
df_anuncios_analise

In [ ]:
orient = 'index'
file_to_save = f'../datasets/inspecao_ecommerce/avaliacao_regulatron/links_amazon_{orient}.json'
df_espatula = df_regulatron.iloc[:,:8]
df_espatula = df_espatula.set_index('url',drop=False)
df_espatula.to_json(file_to_save,orient=orient, force_ascii=False)
df_espatula